In [176]:
import pandas as pd
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
import seaborn as sns
import matplotlib.pyplot as plt
from lazypredict.Supervised import LazyClassifier


import re



In [177]:
data = pd.read_excel(r'Datasets\final_project.ods', engine='odf', dtype='str')
data.head()

,title,location,description,function,industry,career_level
0,Technical Professional Lead - Process,"Houston, TX","Responsible for the study, design, and specifi...",production_manufacturing,Machinery and Industrial Facilities Engineering,senior_specialist_or_project_manager
1,Cnslt - Systems Eng- Midrange 1,"Seattle, WA","Participates in design, development and implem...",information_technology_telecommunications,Financial Services,senior_specialist_or_project_manager
2,SharePoint Developers and Solution Architects,"Dallas, TX",We are currently in need of Developers who can...,consulting,IT Consulting,senior_specialist_or_project_manager
3,Business Information Services - Strategic Acco...,North Carolina,Experian is seeking an experienced Account Exe...,sales,"Security, Risk, Restructuring Consulting",senior_specialist_or_project_manager
4,Strategic Development Director (procurement),"Austin, TX",Â Want to join a world-class global procuremen...,procurement_materials_logistics,Information Technology,bereichsleiter


In [178]:
data['career_level'].value_counts()

career_level
senior_specialist_or_project_manager      4338
manager_team_leader                       2672
bereichsleiter                             960
director_business_unit_leader               70
specialist                                  30
managing_director_small_medium_company       4
Name: count, dtype: int64

In [179]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8074 entries, 0 to 8073
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         8074 non-null   object
 1   location      8074 non-null   object
 2   description   8073 non-null   object
 3   function      8074 non-null   object
 4   industry      8074 non-null   object
 5   career_level  8074 non-null   object
dtypes: object(6)
memory usage: 378.6+ KB


In [180]:
data.dropna(how='any', inplace=True, axis=0)

In [181]:
# df_test = pd.DataFrame(vertorizer.vocabulary_.items(), columns=['key', 'count']).sort_values('count', ascending=False).reset_index(drop=True)
# df_test.head()

In [182]:
def extract_state(x):
    result = re.findall(r'\,\s[A-Z]{2}', x)
    if len(result) > 0:
        return result[0][2:]
    else:
        return x

In [183]:
data['location_state'] = data['location'].apply(extract_state)
data.drop('location', axis=1, inplace=True)
data

,title,description,function,industry,career_level,location_state
0,Technical Professional Lead - Process,"Responsible for the study, design, and specifi...",production_manufacturing,Machinery and Industrial Facilities Engineering,senior_specialist_or_project_manager,TX
1,Cnslt - Systems Eng- Midrange 1,"Participates in design, development and implem...",information_technology_telecommunications,Financial Services,senior_specialist_or_project_manager,WA
2,SharePoint Developers and Solution Architects,We are currently in need of Developers who can...,consulting,IT Consulting,senior_specialist_or_project_manager,TX
3,Business Information Services - Strategic Acco...,Experian is seeking an experienced Account Exe...,sales,"Security, Risk, Restructuring Consulting",senior_specialist_or_project_manager,North Carolina
4,Strategic Development Director (procurement),Â Want to join a world-class global procuremen...,procurement_materials_logistics,Information Technology,bereichsleiter,TX
...,...,...,...,...,...,...
8069,Sales Engineer,You are the primary technical resource for the...,sales,Hardware,senior_specialist_or_project_manager,MA
8070,"Director of Sales, ClosetMaid",Act as the primary contact for Home Depot deci...,sales,Furniture,senior_specialist_or_project_manager,GA
8071,PROPERTY MANAGEMENT: Regional Vice President,"Responsible for the management of office, mult...",finance_accounting,Real Estate Investment Companies,manager_team_leader,CA
8072,Technical Manager SAP Senior,Viseo Business Solution Practice invested in S...,consulting,IT Consulting,senior_specialist_or_project_manager,New York


In [184]:
data['career_level'].nunique()

6

In [185]:
for i in data.columns:
    print(i,":" , data[i].nunique())

title : 6789


description : 7973
function : 19
industry : 352
career_level : 6
location_state : 97


In [186]:
target = 'career_level'
x = data.drop(target, axis=1)
y = data[target]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [187]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(6458, 5) (1615, 5) (6458,) (1615,)


In [188]:
x_train.columns

Index(['title', 'description', 'function', 'industry', 'location_state'], dtype='object')

In [189]:
preprocessor = ColumnTransformer(
    transformers=[
        ('title', TfidfVectorizer(), 'title'),
        ('description', TfidfVectorizer(stop_words='english'), 'description'),
        ('function', OneHotEncoder(handle_unknown='ignore'),['function']),
        ('industry', OneHotEncoder(handle_unknown='ignore'),['industry']),
        ('location_state', OneHotEncoder(handle_unknown='ignore'),['location_state'])
    ]
)

In [190]:
# x_train = preprocessor.fit_transform(x_train)
# x_test = preprocessor.transform(x_test)

In [191]:
# model = SVC()
# model.fit(x_train, y_train)
# y_pred = model.predict(x_test)
# accuracy_score(y_test, y_pred)
# 0.7690402476780186

In [192]:
# model = LogisticRegression()
# model.fit(x_train, y_train)
# y_pred = model.predict(x_test)
# accuracy_score(y_test, y_pred)
# 0.7721362229102167

In [194]:
# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(x_train, x_test, y_train, y_test)
# models

  0%|          | 0/31 [00:00<?, ?it/s]d:\Hutson\learning-materials\AI&ML\Khóa 12-02AIMLDLCV nâng cao\Class\sklearn-env\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
  6%|▋         | 2/31 [00:00<00:11,  2.54it/s]d:\Hutson\learning-materials\AI&ML\Khóa 12-02AIMLDLCV nâng cao\Class\sklearn-env\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
 52%|█████▏    | 16/31 [00:06<00:09,  1.61it/s]d:\Hutson\learning-materials\AI&ML\Khóa 12-02AIMLDLCV nâng cao\Class\sklearn-env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
